In [1]:
from dataclasses import dataclass
from __future__ import annotations
from collections import deque

In [2]:
def get_input(fname="input.txt"):
    with open(fname) as f:
        for line in f.readlines():
            line = line.strip()[:-1]
            container, contains = line.split(" bags contain ")
            if contains == "no other bags":
                yield container, []
                continue
            contained_parts = contains.split(", ")
            contained = []
            for c in contained_parts:
                p = c.split(" ")
                contained.append((int(p[0]), " ".join(p[1:-1])))
            yield container, contained

In [3]:
test_data = list(get_input("test.txt"))

In [4]:
test_data

[('light red', [(1, 'bright white'), (2, 'muted yellow')]),
 ('dark orange', [(3, 'bright white'), (4, 'muted yellow')]),
 ('bright white', [(1, 'shiny gold')]),
 ('muted yellow', [(2, 'shiny gold'), (9, 'faded blue')]),
 ('shiny gold', [(1, 'dark olive'), (2, 'vibrant plum')]),
 ('dark olive', [(3, 'faded blue'), (4, 'dotted black')]),
 ('vibrant plum', [(5, 'faded blue'), (6, 'dotted black')]),
 ('faded blue', []),
 ('dotted black', [])]

In [5]:
@dataclass
class Node:
    color: str
    contains: dict[Node, int]
    contained_in: set[Node]
    
    def __hash__(self):
        return hash(self.color)

In [6]:
def get_bag_types(data):
    make_node = lambda color: Node(color, {}, set())
    BagTypes = {}
    for color, contained in data:
        if color not in BagTypes:
            BagTypes[color] = make_node(color)
        bag = BagTypes[color]
        for qty, c_color in contained:
            if c_color not in BagTypes:
                BagTypes[c_color] = make_node(c_color)
            c_bag = BagTypes[c_color]
            bag.contains[c_bag] = qty
            c_bag.contained_in.add(bag)
    return BagTypes

In [7]:
TestBagTypes = get_bag_types(test_data)

In [8]:
[bag.color for bag in TestBagTypes.values()]

['light red',
 'bright white',
 'muted yellow',
 'dark orange',
 'shiny gold',
 'faded blue',
 'dark olive',
 'vibrant plum',
 'dotted black']

In [9]:
print([c.color for c in TestBagTypes['shiny gold'].contained_in])

['muted yellow', 'bright white']


In [10]:
def solution1(BagTypes, color):
    found: set[Node] = set()
    q = deque(BagTypes[color].contained_in)
    while len(q) > 0:
        n = q.pop()
        found.add(n)
        q.extend(n.contained_in - found)
    return found

In [11]:
len(solution1(TestBagTypes, 'shiny gold'))

4

In [12]:
print([c.color for c in solution1(TestBagTypes, 'shiny gold')])

['muted yellow', 'bright white', 'dark orange', 'light red']


In [13]:
input_data = get_input()

In [14]:
BagTypes = get_bag_types(input_data)

In [15]:
len(solution1(BagTypes, 'shiny gold'))

148

In [16]:
def number_of_bags(BagTypes, color, BagNumbers={}):
    if color in BagNumbers:
        return BagNumbers[color]
    cnt = 0
    for clr, qty in BagTypes[color].contains.items():
        cnt += (1 + number_of_bags(BagTypes, clr.color, BagNumbers)) * qty
    return cnt

In [17]:
number_of_bags(TestBagTypes, 'shiny gold')

32

In [18]:
number_of_bags(BagTypes, 'shiny gold', {})

24867